In [10]:
import requests
import pandas as pd
from itertools import product
import plotly.graph_objects as go

# Hent data fra SSB API
api_url = "https://data.ssb.no/api/v0/no/table/03024/"
query = {
    "query": [],
    "response": {
        "format": "json-stat2"
    }
}
response = requests.post(api_url, json=query)

if response.status_code == 200:
    data = response.json()

    # Trekk ut dimensjoner og verdier
    dimension_names = list(data["dimension"].keys())
    values = data["value"]
    dimensions = [list(data["dimension"][dim]["category"]["label"].values()) for dim in dimension_names]
    all_combinations = list(product(*dimensions))

    df = pd.DataFrame(all_combinations, columns=dimension_names)
    df["Value"] = values

    # Pivotér
    df_pivot = df.pivot_table(
        index="Tid",
        columns=["VareGrupper2", "ContentsCode"],
        values="Value",
        aggfunc="sum"
    ).reset_index()

    # Gi forståelige kolonnenavn
    df_pivot.columns = ["Tid", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]

    # Hent ut år fra Tid (første 4 tegn)
    df_pivot["År"] = df_pivot["Tid"].str[:4].astype(int)

    # Aggregér data per år (gjennomsnitt)
    df_agg = df_pivot.groupby("År", as_index=False).mean(numeric_only=True)

    # Plotly-graf
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Vekt (tonn)"],
        name="Fersk laks - Vekt (tonn)",
        yaxis="y1",
        mode="lines+markers"
    ))

    fig.add_trace(go.Scatter(
        x=df_agg["År"],
        y=df_agg["Fersk laks - Kilospris"],
        name="Fersk laks - Kilospris (kr/kg)",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Fersk laks – Årlig gjennomsnitt av vekt og kilospris",
        xaxis=dict(title="År"),
        yaxis=dict(title="Vekt (tonn)", side="left"),
        yaxis2=dict(title="Kilospris (kr/kg)", overlaying="y", side="right"),
        legend=dict(x=0.01, y=0.99),
        hovermode="x unified",
        template="plotly_white",
        height=600
    )

    fig.show()

else:
    print(f"Feil ved henting av data: {response.status_code}")
    print(response.text)
